# Multiple Regression
Vamos pegar um pequeno conjunto de dados de valores de carros do Blue Book:

In [34]:
import pandas as pd

# URL do arquivo Excel
url = 'http://cdn.sundog-soft.com/Udemy/DataScience/cars.xls'

# Lê o arquivo Excel em um DataFrame do Pandas
df = pd.read_excel(url)

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
print(df.head())


          Price  Mileage   Make    Model      Trim   Type  Cylinder  Liter  \
0  17314.103129     8221  Buick  Century  Sedan 4D  Sedan         6    3.1   
1  17542.036083     9135  Buick  Century  Sedan 4D  Sedan         6    3.1   
2  16218.847862    13196  Buick  Century  Sedan 4D  Sedan         6    3.1   
3  16336.913140    16342  Buick  Century  Sedan 4D  Sedan         6    3.1   
4  16339.170324    19832  Buick  Century  Sedan 4D  Sedan         6    3.1   

   Doors  Cruise  Sound  Leather  
0      4       1      1        1  
1      4       1      1        0  
2      4       1      1        0  
3      4       1      0        0  
4      4       1      0        1  


In [35]:
%matplotlib inline
import pandas as pd
import numpy as np

# Carrega o DataFrame a partir da URL do arquivo Excel
url = 'http://cdn.sundog-soft.com/Udemy/DataScience/cars.xls'
df = pd.read_excel(url)

# Seleciona apenas as colunas 'Mileage' e 'Price' do DataFrame
df1 = df[['Mileage', 'Price']]

# Define os intervalos de bins para agrupamento por Mileage
bins = np.arange(0, 50001, 10000)  # Ajustado para incluir o último valor 50000

# Agrupa os dados por intervalos de Mileage e calcula a média dos preços para cada grupo
groups = df1.groupby(pd.cut(df1['Mileage'], bins, include_lowest=True)).mean()
print(groups.head())

# Plota a média dos preços por intervalo de Mileage
groups['Price'].plot.line()



                         Mileage         Price
Mileage                                       
(-0.001, 10000.0]    5588.629630  24096.714451
(10000.0, 20000.0]  15898.496183  21955.979607
(20000.0, 30000.0]  24114.407104  20278.606252
(30000.0, 40000.0]  33610.338710  19463.670267
(40000.0, 50000.0]  43159.200000  30802.690000


C:\Users\jonin\AppData\Local\Temp\ipykernel_29852\1610886733.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = df1.groupby(pd.cut(df1['Mileage'], bins, include_lowest=True)).mean()


<Axes: xlabel='Mileage'>

Podemos usar pandas para dividir essa matriz nos vetores de características nos quais estamos interessados ​​e no valor que estamos tentando prever. Observe como estamos evitando a marca e o modelo; regressões não funcionam bem com valores ordinais, a menos que você possa convertê-los em alguma ordem numérica que faça sentido de alguma forma. Vamos dimensionar nossos dados de recursos no mesmo intervalo para que possamos comparar facilmente os coeficientes obtidos.

In [36]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Carrega o DataFrame a partir da URL do arquivo Excel
url = 'http://cdn.sundog-soft.com/Udemy/DataScience/cars.xls'
df = pd.read_excel(url)

# Inicializa o StandardScaler
scale = StandardScaler()

# Seleciona as colunas de características (features) e a variável dependente (target)
X = df[['Mileage', 'Cylinder', 'Doors']]
y = df['Price']

# Padroniza (escala) as colunas de características
X[['Mileage', 'Cylinder', 'Doors']] = scale.fit_transform(X[['Mileage', 'Cylinder', 'Doors']])

# Adiciona uma coluna constante para termo de interceptação
X = sm.add_constant(X)

# Ajusta o modelo de regressão linear usando Ordinary Least Squares (OLS)
est = sm.OLS(y, X).fit()

# Imprime o resumo estatístico do modelo
print(est.summary())



                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     150.0
Date:                Thu, 26 Sep 2024   Prob (F-statistic):           3.95e-77
Time:                        09:48:50   Log-Likelihood:                -8356.7
No. Observations:                 804   AIC:                         1.672e+04
Df Residuals:                     800   BIC:                         1.674e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.134e+04    279.405     76.388      0.0

C:\Users\jonin\AppData\Local\Temp\ipykernel_29852\1003429986.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Mileage', 'Cylinder', 'Doors']] = scale.fit_transform(X[['Mileage', 'Cylinder', 'Doors']])


A tabela de coeficientes acima nos dá os valores para inserir em uma equação da forma: B0 + B1 * Quilometragem + B2 * cilindros + B3 * portas 
Neste exemplo, fica bem claro que o número de cilindros é mais importante do que qualquer coisa baseada nos coeficientes.

Poderíamos ter descoberto isso antes?

In [37]:
y.groupby(df.Doors).mean()

Doors
2    23807.135520
4    20580.670749
Name: Price, dtype: float64

Surpreendentemente, mais portas não significam um preço mais elevado! (Talvez isso implique um carro esporte em alguns casos?) Portanto, não é surpreendente que seja inútil como preditor aqui. No entanto, este é um conjunto de dados muito pequeno, por isso não podemos realmente ler muito significado nele.

Como você usaria isso para fazer uma previsão real? Comece dimensionando suas múltiplas variáveis ​​de recursos na mesma escala usada para treinar o modelo e, em seguida, basta chamar est.predict() nos recursos dimensionados:

In [39]:
scaled = scale.transform([[45000, 8, 4]])
scaled = np.insert(scaled[0], 0, 1) #Need to add that constant column in again.
print(scaled)
predicted = est.predict(scaled)
print(predicted)

[1.         3.07256589 1.96971667 0.55627894]
[27658.15707316]


C:\Users\jonin\miniconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [38]:
# Dados de entrada escalados e com a constante adicionada
scaled = scale.transform([[45000, 8, 4]])
scaled_with_const = np.insert(scaled[0], 0, 1)

# Realiza a previsão usando o modelo ajustado
predicted = est.predict(scaled_with_const.reshape(1, -1))

print("Dados de entrada escalados com constante:")
print(scaled_with_const)
print("\nPrevisão do preço do carro:")
print(predicted)


Dados de entrada escalados com constante:
[1.         3.07256589 1.96971667 0.55627894]

Previsão do preço do carro:
[27658.15707316]


C:\Users\jonin\miniconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [40]:
# Criar um DataFrame para os novos dados com os mesmos nomes de colunas
new_data = pd.DataFrame([[45000, 8, 4]], columns=['Mileage', 'Cylinder', 'Doors'])

# Escalar os novos dados
scaled = scale.transform(new_data)

# Adicionar a coluna constante (para o intercepto)
scaled_with_const = np.insert(scaled[0], 0, 1)

# Realiza a previsão usando o modelo ajustado
predicted = est.predict(scaled_with_const.reshape(1, -1))

print("Dados de entrada escalados com constante:")
print(scaled_with_const)
print("\nPrevisão do preço do carro:")
print(predicted)


Dados de entrada escalados com constante:
[1.         3.07256589 1.96971667 0.55627894]

Previsão do preço do carro:
[27658.15707316]


## Activity
Mexa com os dados de entrada falsos e veja se você consegue criar uma influência mensurável do número de portas no preço. Divirta-se com isso - por que parar nas 4 portas?